In [41]:
import pickle
import requests
import tenseal as ts
from cc_he_utils import json_serialize, json_deserialize, load_test_data

In [42]:
def setup_context():
    # TenSEAL CKKS HE encryption scheme context setup
    bits_scale = 50
    coeff_mod_bit_sizes = [60, bits_scale, bits_scale, bits_scale, 60]
    polynomial_modulus_degree = 8192*2

    # Create context
    context = ts.context(ts.SCHEME_TYPE.CKKS, polynomial_modulus_degree, coeff_mod_bit_sizes=coeff_mod_bit_sizes)
    # Set global scale
    context.global_scale = 2**bits_scale
    # Generate galois keys required for matmul in ckks_vector
    context.generate_galois_keys()
    return context

In [43]:
xtest, ytest = load_test_data()
context = setup_context()

In [48]:
%%time
enc_input = ts.ckks_vector(context, xtest[0])

CPU times: user 18.6 ms, sys: 420 µs, total: 19 ms
Wall time: 18.5 ms


In [49]:
%%time
request_data = {
    'input': json_serialize(enc_input.serialize()),
    'context': json_serialize(context.serialize()),
}

CPU times: user 582 ms, sys: 264 ms, total: 845 ms
Wall time: 839 ms


In [50]:
%%time
response = requests.post('http://134.226.86.101:8000/nn', json=request_data, timeout=10).json()

CPU times: user 938 ms, sys: 210 ms, total: 1.15 s
Wall time: 2.76 s


In [51]:
%%time
enc_x = ts.ckks_vector_from(context, json_deserialize(response['result']))

CPU times: user 963 µs, sys: 0 ns, total: 963 µs
Wall time: 974 µs


In [52]:
%%time
enc_x.decrypt()

CPU times: user 0 ns, sys: 6.32 ms, total: 6.32 ms
Wall time: 5.59 ms


[-4.287199275793964]